In [1]:
import pickle
import pandas as pd
from scripts.Embedder import Embedder, tfidfEmbedder
from scripts.OccupationPreprocessor import OccupationPreprocessor
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gradlab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open('first_dig_tfidf_clfs.pkl', 'rb') as f:
    clf1 = pickle.load(f)
    
with open('second_third_fourth_dig_tfidf_clfs.pkl', 'rb') as f2:
    clf2 = pickle.load(f2)

In [3]:
TEST_FILE = 2

In [4]:
if TEST_FILE == 0:
    tfidf_test_set = OccupationPreprocessor.prepare_df(
        './Data/ATP_gold_standard.xlsx',
        input_column='CURRENT_JOB_TITLE',
        code_column='NOC code by PATH',
        n_digits=4
    )
    
    NOCv5 = OccupationPreprocessor.prepare_df(
        './Data/ATP_gold_standard.xlsx',
        input_column='NOC code by PATH',
        code_column='V5_NOC',
        n_digits=4
    )
    
    NOCv6 = OccupationPreprocessor.prepare_df(
        './Data/ATP_gold_standard.xlsx',
        input_column='NOC code by PATH',
        code_column='V6_NOC',
        n_digits=4
    )
    
elif TEST_FILE == 1:
    tfidf_test_set = pd.DataFrame(
        pd.read_csv('./Data/tfidf_test_set.csv')
    )
    
# A-B test set
elif TEST_FILE == 2:
    tfidf_test_set = pd.DataFrame(
        pd.read_csv('overlap_test_set_v4_acanoc.csv')
    )
    
tfidf_train_set = pd.DataFrame(
    pd.read_csv('./Data/tfidf_train_set.csv')
)

In [5]:
embedder = tfidfEmbedder()

Model loaded from ./vectorizer.joblib


In [6]:
sample_size = 500 if TEST_FILE == 0 else 12000
sample_pipeline_df = tfidf_test_set.sample(sample_size, random_state=123)#.drop_duplicates()

### Metrics for alternative classifiers

In [7]:
from sklearn.metrics import accuracy_score, f1_score
def clean_noc_double_codes(row, column):
    if ',' in row[column]:
        row[column] = str(row[column].strip('\'').split(',')[1])
    return row

if TEST_FILE == 0:
    NOCv5 = NOCv5.apply(clean_noc_double_codes, axis =1 , args = ('input',))
    NOCv6 = NOCv6.apply(clean_noc_double_codes, axis =1 , args = ('input',))
    print('NOCv5 Ensemble metrics total. Accuracy:{}, f1:{}\n'.format(
        accuracy_score(NOCv5['input'].astype(int), NOCv5['code'].astype(int)), 
        f1_score(NOCv5['input'].astype(int), NOCv5['code'].astype(int), average = 'macro')
    ))
    print('NOCv6 Ensemble metrics total. Accuracy:{}, f1:{}\n'.format(
        accuracy_score(NOCv6['input'].astype(int), NOCv6['code'].astype(int)), 
        f1_score(NOCv6['input'].astype(int), NOCv6['code'].astype(int), average = 'macro')
    ))
elif TEST_FILE == 2:
    print(
        "Accuracy to Beat:",
        accuracy_score(
            sample_pipeline_df['code'], 
            sample_pipeline_df['v4_pred']
        )
    )

Accuracy to Beat: 0.5459166666666667


### Check exact matches

In [8]:
tfidf_test_vectors = embedder.embed(sample_pipeline_df['input'])

In [9]:
tfidf_test_df = pd.DataFrame({
    'svm_pred':clf1['SVM'].predict(tfidf_test_vectors),
    'rf_pred':clf1['RF'].predict(tfidf_test_vectors),
    'knn_pred':clf1['KNN'].predict(tfidf_test_vectors),
    'code':sample_pipeline_df['code'].astype(str)
})

In [10]:
tfidf_test_df['exact_match'] = sample_pipeline_df['input'].apply(embedder.check_exact_match, args=(embedder.train_database,))

In [11]:
tfidf_test_df['p_all'] = tfidf_test_df.apply(tfidfEmbedder.ensemble_predict, axis = 1, args = (
    ['rf_pred','svm_pred','knn_pred'], 'svm_pred',
))

In [12]:
tfidf_test_df['vectors'] = tfidf_test_vectors.toarray().tolist()

In [13]:
# put in pipeline.py
def pipeline(row):
    np_array = np.array(row['vectors']).reshape(1, -1)
    p_1 = row['p_all']
    row['svm_pred_234'] = clf2[p_1]['SVM'].predict(np_array)[0]
    row['rf_pred_234'] = clf2[p_1]['RF'].predict(np_array)[0]
    row['knn_pred_234'] = clf2[p_1]['KNN'].predict(np_array)[0]
    return row

In [14]:
import time
start_time = time.time()
tfidf_test_df = tfidf_test_df.apply(pipeline, axis = 1)
elapsed = time.time()-start_time
print('time to predict on {0} samples: {1:.2f} seconds ({2:.3f} s/sample)'.format(sample_pipeline_df.shape[0],
                                                         elapsed, 
                                                        elapsed/sample_pipeline_df.shape[0])
     )

time to predict on 12000 samples: 2812.53 seconds (0.234 s/sample)


In [15]:
tfidf_test_df['p_all_234'] = tfidf_test_df.apply(tfidfEmbedder.ensemble_predict, axis = 1, args = (
    ['svm_pred_234','rf_pred_234','knn_pred_234'], 'knn_pred_234',
))

### performance w/o exact match

In [16]:
from sklearn.metrics import accuracy_score, f1_score

for classifier in ['knn','svm', 'rf']:
    print('{} acc:{}, f1-macro:{}'.format(classifier.upper(), 
                                    accuracy_score(
                                        tfidf_test_df['{}_pred_234'.format(classifier)].astype(int), 
                                        tfidf_test_df['code'].astype(int)
                                    ),
                                    f1_score(
                                        tfidf_test_df['{}_pred_234'.format(classifier)].astype(int),
                                        tfidf_test_df['code'].astype(int), average = 'macro')
                                   )
     )
print('Ensemble metrics total. Accuracy:{}, f1:{}\n'.format(
    accuracy_score(tfidf_test_df['p_all_234'].astype(int), tfidf_test_df['code'].astype(int)), 
    f1_score(tfidf_test_df['p_all_234'].astype(int), tfidf_test_df['code'].astype(int), average = 'macro')
))

KNN acc:0.6139166666666667, f1-macro:0.4046073687683105
SVM acc:0.6251666666666666, f1-macro:0.4411848648665285
RF acc:0.6388333333333334, f1-macro:0.43846144747197313
Ensemble metrics total. Accuracy:0.6435, f1:0.43977552074060466



### With exact match added

In [17]:
tfidf_test_df['Hybrid_pred_exact_match'] = tfidf_test_df.apply(
    lambda row : row['p_all_234'] if row['exact_match'] == -1 else row['exact_match'], axis = 1
)
print('Ensemble metrics after checking for exact match total. Accuracy:{}, f1:{}\n'.format(
    accuracy_score(tfidf_test_df['Hybrid_pred_exact_match'].astype(int), tfidf_test_df['code'].astype(int)), 
    f1_score(tfidf_test_df['Hybrid_pred_exact_match'].astype(int), tfidf_test_df['code'].astype(int), average = 'macro')
))

Ensemble metrics after checking for exact match total. Accuracy:0.6439166666666667, f1:0.44015503118631666



In [18]:
tfidf_test_df.drop(columns =['vectors'])

,svm_pred,rf_pred,knn_pred,code,exact_match,p_all,svm_pred_234,rf_pred_234,knn_pred_234,p_all_234,Hybrid_pred_exact_match
30811,6,9,6,6731,-1,6,6731,6731,6731,6731,6731
21792,8,7,8,8222,-1,8,8211,8221,8221,8221,8221
32869,8,9,9,8611,-1,9,9619,9619,9611,9619,9619
35132,0,0,0,601,-1,0,125,911,122,122,122
29989,0,9,0,1123,-1,0,11,821,124,124,124
...,...,...,...,...,...,...,...,...,...,...,...
7799,3,3,3,3012,-1,3,3012,3012,3012,3012,3012
20459,4,4,4,414,-1,4,4161,4164,4164,4164,4164
35252,7,7,7,7205,-1,7,7302,7302,7205,7302,7302
23258,1,9,1,1111,-1,1,1111,1111,1111,1111,1111
